In [2]:
import os
import sys
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
sys.path.append(r"D:/My File/SMU/QF 635/MMAT")
from config.load_env import load_keys
from signal_logger import SignalHistoryLogger

In [1]:
"""
from binance.client import Client
keys = load_keys()
print("Loaded keys:", keys)
client = Client(keys['api_key'], keys['secret_key'])
"""

'\nfrom binance.client import Client\nkeys = load_keys()\nprint("Loaded keys:", keys)\nclient = Client(keys[\'api_key\'], keys[\'secret_key\'])\n'

In [8]:
logger = SignalHistoryLogger(filename="D:/My File/SMU/QF 635/MMAT/signal_history_xie.csv")

In [10]:
def fetch_ohlcv_data(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_5MINUTE, limit=1000):
    klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_volume', 'taker_buy_quote_volume', 'ignore'
    ])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df

In [12]:
def load_local_csv_data(csv_path):
    try:
        df = pd.read_csv(csv_path, index_col='timestamp', parse_dates=True)
        df = df[['open', 'high', 'low', 'close', 'volume']].copy()
        print(f"Total K-lines loaded from CSV: {len(df)}")
        print(df.head())
        return df
    except FileNotFoundError:
        print(f"CSV file '{csv_path}' not found.")
        return None

In [14]:
def calculate_indicators(df):
    df['MA5'] = talib.SMA(df['close'], timeperiod=5)
    df['MA20'] = talib.SMA(df['close'], timeperiod=20)
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['mean_ATR'] = df['ATR'].rolling(window=20).mean()
    df['Volume_MA20'] = df['volume'].rolling(window=20).mean()
    macd, macdsignal, macdhist = talib.MACD(df['close'], fastperiod=12, slowperiod=26, signalperiod=9)
    df['MACD'] = macd
    df['MACD_signal'] = macdsignal
    return df
    
import talib

def calculate_patterns(df):
    """
    Detect selected high-accuracy candlestick patterns ONLY on the latest completed candle.
    """
    patterns = {
        'Hammer': talib.CDLHAMMER,
        'InvertedHammer': talib.CDLINVERTEDHAMMER,
        'BullishEngulfing': lambda o, h, l, c: np.where(talib.CDLENGULFING(o, h, l, c) == 100, 100, 0),
        'PiercingLine': talib.CDLPIERCING,
        'MorningStar': talib.CDLMORNINGSTAR,
        'DragonflyDoji': talib.CDLDRAGONFLYDOJI,
        'LongLine': talib.CDLLONGLINE,
        'HangingMan': talib.CDLHANGINGMAN,
        'ShootingStar': talib.CDLSHOOTINGSTAR,
        'BearishEngulfing': lambda o, h, l, c: np.where(talib.CDLENGULFING(o, h, l, c) == -100, -100, 0),
        'DarkCloudCover': talib.CDLDARKCLOUDCOVER,
        'EveningDojiStar': talib.CDLEVENINGDOJISTAR,
        'EveningStar': talib.CDLEVENINGSTAR,
        'GravestoneDoji': talib.CDLGRAVESTONEDOJI,
        'ThreeLineStrike': talib.CDL3LINESTRIKE,
    }

    for name in patterns.keys():
        df[name] = 0

    i = len(df) - 2
    for name, func in patterns.items():
        result = func(df['open'], df['high'], df['low'], df['close'])
        value = result[i] if isinstance(result, np.ndarray) else result.iloc[i]
        df.at[df.index[i], name] = value

    for name in patterns.keys():
        print(f"{name} at [-2]: {df[name].iloc[-2]}")

    bullish_patterns = ['Hammer', 'InvertedHammer', 'BullishEngulfing', 'PiercingLine','MorningStar', 'DragonflyDoji', 'LongLine', 'ThreeLineStrike']
    bearish_patterns = ['HangingMan', 'ShootingStar', 'BearishEngulfing', 'DarkCloudCover','EveningDojiStar', 'EveningStar', 'GravestoneDoji','LongLine','ThreeLineStrike']
    return df, patterns, bullish_patterns, bearish_patterns

def generate_signals(df, patterns, window=1):
    """
    Generate candlestick pattern signals using only the latest closed candle (index -2).
    Avoids retroactive rewriting of historical signals.
    """
    for name in patterns.keys():
        df[f'Signal_{name}'] = 0
        df[f'Direction_{name}'] = 'NONE'

    i = len(df) - 2
    for name in patterns.keys():
        value = df[name].iloc[i]

        if name in ['BullishEngulfing', 'ThreeLineStrike'] and value == 100:
            df.loc[df.index[i], f'Signal_{name}'] = 1
            df.loc[df.index[i], f'Direction_{name}'] = 'UP'
        elif name in ['Hammer', 'InvertedHammer', 'PiercingLine','MorningStar', 'DragonflyDoji', 'LongLine'] and value > 0:
            df.loc[df.index[i], f'Signal_{name}'] = 1
            df.loc[df.index[i], f'Direction_{name}'] = 'UP'
        elif name in ['BearishEngulfing','ThreeLineStrike'] and value == -100:
            df.loc[df.index[i], f'Signal_{name}'] = -1
            df.loc[df.index[i], f'Direction_{name}'] = 'DOWN'
        elif name in ['HangingMan', 'ShootingStar', 'DarkCloudCover','EveningDojiStar', 'EveningStar','LongLine'] and value < 0:
            df.loc[df.index[i], f'Signal_{name}'] = -1
            df.loc[df.index[i], f'Direction_{name}'] = 'DOWN'
        elif name == 'GravestoneDoji' and value == 100:
            df.loc[df.index[i], f'Signal_{name}'] = -1
            df.loc[df.index[i], f'Direction_{name}'] = 'DOWN'

    return df

策略1-5，最初版本

In [17]:
import talib

def generate_xie1_signals(df, logger=None):
    """
    Apply five technical signal strategies on the most recent completed candle (i = -2).
    Adds 'bullish_combined' or 'bearish_combined' markers to the DataFrame.

    Args:
        df (pd.DataFrame): DataFrame with technical indicators and patterns.
        logger (SignalHistoryLogger, optional): Used for logging signal records.
    """
    i = len(df) - 2  # Evaluate the most recent *completed* candle
    df['bullish_combined'] = 0
    df['bearish_combined'] = 0

    # === Strategy 1: RSI Divergence + Hammer ===
    if (
        df['RSI'].iloc[i] > df['RSI'].iloc[i-1] and
        df['close'].iloc[i] < df['close'].iloc[i-1] and
        df['Hammer'].iloc[i] > 0
    ):
        df.loc[df.index[i], 'bullish_combined'] = 1
        if logger:
            logger.add_signal("bullish", df.index[i], df['close'].iloc[i], "RSI Divergence + Hammer")

    # === Strategy 2: MACD Flip + ShootingStar ===
    hist = df['MACD'] - df['MACD_signal']
    if (
        hist.iloc[i] < 0 and hist.iloc[i-1] > 0 and
        df['ShootingStar'].iloc[i] < 0
    ):
        df.loc[df.index[i], 'bearish_combined'] = -1
        if logger:
            logger.add_signal("bearish", df.index[i], df['close'].iloc[i], "MACD Flip + ShootingStar")

    # === Strategy 3: Bollinger Rebound + RSI + BullishEngulfing ===
    upper, middle, lower = talib.BBANDS(df['close'], timeperiod=20)
    if (
        df['close'].iloc[i-1] < lower[i-1] and
        df['close'].iloc[i] > lower[i] and
        df['RSI'].iloc[i] > 50 and
        df['BullishEngulfing'].iloc[i] == 100
    ):
        df.loc[df.index[i], 'bullish_combined'] = 1
        if logger:
            logger.add_signal("bullish", df.index[i], df['close'].iloc[i], "Bollinger Rebound + RSI + BullishEngulfing")

    # === Strategy 4: ATR Spike + Volume Surge + RSI < 30 ===
    if (
        df['ATR'].iloc[i] > 1.3 * df['mean_ATR'].iloc[i] and
        df['volume'].iloc[i] > 1.5 * df['Volume_MA20'].iloc[i] and
        df['RSI'].iloc[i] < 30
    ):
        df.loc[df.index[i], 'bullish_combined'] = 1
        if logger:
            logger.add_signal("bullish", df.index[i], df['close'].iloc[i], "ATR Spike + Volume Surge + RSI<30")

    # === Strategy 5: RSI > 70 + MACD Cross + Bearish Engulfing ===
    if (
        df['RSI'].iloc[i] > 70 and
        df['MACD'].iloc[i] < df['MACD_signal'].iloc[i] and
        df['BearishEngulfing'].iloc[i] == -100
    ):
        df.loc[df.index[i], 'bearish_combined'] = -1
        if logger:
            logger.add_signal("bearish", df.index[i], df['close'].iloc[i], "RSI>70 + MACD Cross + BearishEngulfing")

    return df

In [19]:
import pandas as pd
import numpy as np
import talib
from tqdm import tqdm

# === Step 1: Load Data ===
df = pd.read_csv("BTCUSDT_1min_2024-05-01_to_2025-05-01.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# === Step 2: Technical Indicators & Patterns ===
df['MA5'] = talib.SMA(df['close'], 5)
df['MA20'] = talib.SMA(df['close'], 20)
df['RSI'] = talib.RSI(df['close'], 14)
df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], 14)
df['mean_ATR'] = df['ATR'].rolling(20).mean()
df['Volume_MA20'] = df['volume'].rolling(20).mean()
macd, macdsignal, _ = talib.MACD(df['close'], 12, 26, 9)
df['MACD'] = macd
df['MACD_signal'] = macdsignal
upper, middle, lower = talib.BBANDS(df['close'], 20)

df['Hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
df['ShootingStar'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
df['BullishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
df['BearishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']) * -1

# === Step 3: Strategy Backtesting ===
results = {
    'RSI Divergence + Hammer': {'total': 0, 'hit': 0},
    'MACD Flip + ShootingStar': {'total': 0, 'hit': 0},
    'Bollinger Rebound + RSI + Engulfing': {'total': 0, 'hit': 0},
    'ATR Spike + Volume Surge + RSI<30': {'total': 0, 'hit': 0},
    'RSI>70 + MACD Cross + BearishEngulfing': {'total': 0, 'hit': 0}
}

hist = df['MACD'] - df['MACD_signal']

for i in tqdm(range(1, len(df) - 1), desc="Backtesting..."):
    next_return = (df['close'].iat[i+1] - df['close'].iat[i]) / df['close'].iat[i]

    # Strategy 1: RSI up vs. price down + Hammer
    if df['RSI'].iat[i] > df['RSI'].iat[i-1] and df['close'].iat[i] < df['close'].iat[i-1] and df['Hammer'].iat[i] > 0:
        results['RSI Divergence + Hammer']['total'] += 1
        if next_return > 0:
            results['RSI Divergence + Hammer']['hit'] += 1

    # Strategy 2: MACD Histogram flips + ShootingStar
    if hist.iat[i] < 0 and hist.iat[i-1] > 0 and df['ShootingStar'].iat[i] < 0:
        results['MACD Flip + ShootingStar']['total'] += 1
        if next_return < 0:
            results['MACD Flip + ShootingStar']['hit'] += 1

    # Strategy 3: Close[i-1] < LowerBand & Close[i] > LowerBand + RSI>50 + BullishEngulfing
    if df['close'].iat[i-1] < lower[i-1] and df['close'].iat[i] > lower[i] and df['RSI'].iat[i] > 50 and df['BullishEngulfing'].iat[i] == 100:
        results['Bollinger Rebound + RSI + Engulfing']['total'] += 1
        if next_return > 0:
            results['Bollinger Rebound + RSI + Engulfing']['hit'] += 1

    # Strategy 4: ATR > 1.3×mean + Volume > 1.5×mean + RSI < 30
    if (
        df['ATR'].iat[i] > df['mean_ATR'].iat[i] * 1.3 and
        df['volume'].iat[i] > df['Volume_MA20'].iat[i] * 1.5 and
        df['RSI'].iat[i] < 30
    ):
        results['ATR Spike + Volume Surge + RSI<30']['total'] += 1
        if next_return > 0:
            results['ATR Spike + Volume Surge + RSI<30']['hit'] += 1

    # Strategy 5: RSI > 70 + MACD cross down + BearishEngulfing
    if (
        df['RSI'].iat[i] > 70 and
        df['MACD'].iat[i] < df['MACD_signal'].iat[i] and
        df['BearishEngulfing'].iat[i] == -100
    ):
        results['RSI>70 + MACD Cross + BearishEngulfing']['total'] += 1
        if next_return < 0:
            results['RSI>70 + MACD Cross + BearishEngulfing']['hit'] += 1

# === Step 4: Print Summary ===
summary = []
for name, res in results.items():
    total = res['total']
    hit = res['hit']
    acc = hit / total if total > 0 else np.nan
    summary.append({
        'Strategy': name,
        'Signal Count': total,
        'Correct Predictions': hit,
        'Accuracy': f"{acc:.2%}" if not np.isnan(acc) else "N/A"
    })

summary_df = pd.DataFrame(summary)
print("\n=== Backtest Summary ===")
print(summary_df)

Backtesting...:   0%|          | 0/525998 [00:00<?, ?it/s]C:\Users\XIE\AppData\Local\Temp\ipykernel_35392\1689665838.py:55: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df['close'].iat[i-1] < lower[i-1] and df['close'].iat[i] > lower[i] and df['RSI'].iat[i] > 50 and df['BullishEngulfing'].iat[i] == 100:
Backtesting...: 100%|██████████| 525998/525998 [01:34<00:00, 5553.93it/s]


=== Backtest Summary ===
                                 Strategy  Signal Count  Correct Predictions  \
0                 RSI Divergence + Hammer             0                    0   
1                MACD Flip + ShootingStar            24                   14   
2     Bollinger Rebound + RSI + Engulfing            60                   35   
3       ATR Spike + Volume Surge + RSI<30          2686                 1476   
4  RSI>70 + MACD Cross + BearishEngulfing            24                   10   

  Accuracy  
0      N/A  
1   58.33%  
2   58.33%  
3   54.95%  
4   41.67%  


策略1-5，改进版本（放宽限制）

In [84]:
def generate_xie_signals(df):
    i = len(df) - 2
    df['bullish_combined'] = 0
    df['bearish_combined'] = 0

    # 策略 1：RSI 连续上升 + Hammer
    if (
        df['RSI'].iloc[i] > df['RSI'].iloc[i-1] > df['RSI'].iloc[i-2] and
        df['Hammer'].iloc[i] > 0
    ):
        df.loc[df.index[i], 'bullish_combined'] = 1
        logger.add_signal("bullish", df.index[i], df['close'].iloc[i], "RSI up 3 bars + Hammer")

    # 策略 2：MACD < Signal + ShootingStar
    hist = df['MACD'] - df['MACD_signal']
    if (
        df['MACD'].iloc[i] < df['MACD_signal'].iloc[i] and
        df['ShootingStar'].iloc[i] < 0
    ):
        df.loc[df.index[i], 'bearish_combined'] = -1
        logger.add_signal("bearish", df.index[i], df['close'].iloc[i], "MACD < Signal + ShootingStar")

    # 策略 3：Close near lower BB + RSI > 45 + BullishEngulfing
    upper, middle, lower = talib.BBANDS(df['close'], timeperiod=20)
    if (
        df['close'].iloc[i] < lower[i] * 1.01 and
        df['RSI'].iloc[i] > 45 and
        df['BullishEngulfing'].iloc[i] == 100
    ):
        df.loc[df.index[i], 'bullish_combined'] = 1
        logger.add_signal("bullish", df.index[i], df['close'].iloc[i], "Near Lower BB + RSI>45 + BullishEngulfing")

    # 策略 4：ATR > 1.1 × 平均 + Volume > 1.2 × 均值 + RSI < 35
    if (
        df['ATR'].iloc[i] > df['mean_ATR'].iloc[i] * 1.1 and
        df['volume'].iloc[i] > df['Volume_MA20'].iloc[i] * 1.2 and
        df['RSI'].iloc[i] < 35
    ):
        df.loc[df.index[i], 'bullish_combined'] = 1
        logger.add_signal("bullish", df.index[i], df['close'].iloc[i], "ATR↑ + Vol↑ + RSI<35")

    # 策略 5：RSI > 65 + MACD < Signal + BearishEngulfing
    if (
        df['RSI'].iloc[i] > 65 and
        df['MACD'].iloc[i] < df['MACD_signal'].iloc[i] and
        df['BearishEngulfing'].iloc[i] == -100
    ):
        df.loc[df.index[i], 'bearish_combined'] = -1
        logger.add_signal("bearish", df.index[i], df['close'].iloc[i], "RSI>65 + MACD<Signal + BearishEngulfing")

    return df

In [20]:
import pandas as pd
import numpy as np
import talib
from tqdm import tqdm

# === Step 1: Read historical data ===
df = pd.read_csv("BTCUSDT_1min_2024-05-01_to_2025-05-01.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# === Step 2: Calculation of technical indicators and patterns ===
df['MA5'] = talib.SMA(df['close'], 5)
df['MA20'] = talib.SMA(df['close'], 20)
df['RSI'] = talib.RSI(df['close'], 14)
df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], 14)
df['mean_ATR'] = df['ATR'].rolling(20).mean()
df['Volume_MA20'] = df['volume'].rolling(20).mean()
macd, macdsignal, _ = talib.MACD(df['close'], 12, 26, 9)
df['MACD'] = macd
df['MACD_signal'] = macdsignal
upper, middle, lower = talib.BBANDS(df['close'], 20)

df['Hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
df['ShootingStar'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
df['BullishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
df['BearishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']) * -1

# === Step 3: Evaluate each of the five strategies separately ===
results = {
    'RSI Divergence + Hammer': {'total': 0, 'hit': 0},
    'MACD Flip + ShootingStar': {'total': 0, 'hit': 0},
    'Bollinger Rebound + RSI + Engulfing': {'total': 0, 'hit': 0},
    'ATR Spike + Volume Surge + RSI<30': {'total': 0, 'hit': 0},
    'RSI>70 + MACD Cross + BearishEngulfing': {'total': 0, 'hit': 0}
}

hist = df['MACD'] - df['MACD_signal']

for i in tqdm(range(2, len(df) - 1), desc="Backtesting..."):
    next_return = (df['close'].iat[i+1] - df['close'].iat[i]) / df['close'].iat[i]
    hist = df['MACD'] - df['MACD_signal']

    # Strategy 1: RSI Continuous rise + Hammer
    if df['RSI'].iat[i] > df['RSI'].iat[i-1] > df['RSI'].iat[i-2] and df['Hammer'].iat[i] > 0:
        results['RSI Divergence + Hammer']['total'] += 1
        if next_return > 0:
            results['RSI Divergence + Hammer']['hit'] += 1

    # Strategy 2: MACD < Signal + ShootingStar
    if df['MACD'].iat[i] < df['MACD_signal'].iat[i] and df['ShootingStar'].iat[i] < 0:
        results['MACD Flip + ShootingStar']['total'] += 1
        if next_return < 0:
            results['MACD Flip + ShootingStar']['hit'] += 1

    # Strategy 3: The closing price is close to the lower Bollinger bands + RSI > 45 + BullishEngulfing
    if df['close'].iat[i] < lower[i] * 1.01 and df['RSI'].iat[i] > 45 and df['BullishEngulfing'].iat[i] == 100:
        results['Bollinger Rebound + RSI + Engulfing']['total'] += 1
        if next_return > 0:
            results['Bollinger Rebound + RSI + Engulfing']['hit'] += 1

    # Strategy 4: ATR > 1.1 × mean value + Volume > 1.2 × mean value + RSI < 35
    if (
        df['ATR'].iat[i] > df['mean_ATR'].iat[i] * 1.1 and
        df['volume'].iat[i] > df['Volume_MA20'].iat[i] * 1.2 and
        df['RSI'].iat[i] < 35
    ):
        results['ATR Spike + Volume Surge + RSI<30']['total'] += 1
        if next_return > 0:
            results['ATR Spike + Volume Surge + RSI<30']['hit'] += 1

    # Strategy 5: RSI > 65 + MACD < Signal + BearishEngulfing
    if (
        df['RSI'].iat[i] > 65 and
        df['MACD'].iat[i] < df['MACD_signal'].iat[i] and
        df['BearishEngulfing'].iat[i] == -100
    ):
        results['RSI>70 + MACD Cross + BearishEngulfing']['total'] += 1
        if next_return < 0:
            results['RSI>70 + MACD Cross + BearishEngulfing']['hit'] += 1

# === Step 4: Summarize and output ===
summary = []
for name, res in results.items():
    total = res['total']
    hit = res['hit']
    acc = hit / total if total > 0 else np.nan
    summary.append({
        'Strategy': name,
        'Signal Count': total,
        'Correct Predictions': hit,
        'Accuracy': f"{acc:.2%}"
    })

# Print result
summary_df = pd.DataFrame(summary)
print("\n=== Backtest Summary ===")
print(summary_df)

Backtesting...:   0%|          | 0/525997 [00:00<?, ?it/s]C:\Users\XIE\AppData\Local\Temp\ipykernel_16136\1613599437.py:56: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if df['close'].iat[i] < lower[i] * 1.01 and df['RSI'].iat[i] > 45 and df['BullishEngulfing'].iat[i] == 100:
Backtesting...: 100%|██████████| 525997/525997 [35:16<00:00, 248.57it/s]


=== Backtest Summary ===
                                 Strategy  Signal Count  Correct Predictions  \
0                 RSI Divergence + Hammer           992                  482   
1                MACD Flip + ShootingStar          1199                  645   
2     Bollinger Rebound + RSI + Engulfing         10520                 5418   
3       ATR Spike + Volume Surge + RSI<30         11660                 6114   
4  RSI>70 + MACD Cross + BearishEngulfing           135                   70   

  Accuracy  
0   48.59%  
1   53.79%  
2   51.50%  
3   52.44%  
4   51.85%  


组合策略（单个策略未放宽）（1+3，2+5，2+4，3+4，1+2+4）

In [30]:
import pandas as pd
import numpy as np
import talib
from tqdm import tqdm

# Load data
df = pd.read_csv("BTCUSDT_1min_2024-05-01_to_2025-05-01.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Technical indicators
df['RSI'] = talib.RSI(df['close'], 14)
df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], 14)
df['mean_ATR'] = df['ATR'].rolling(20).mean()
df['Volume_MA20'] = df['volume'].rolling(20).mean()
macd, macdsignal, _ = talib.MACD(df['close'], 12, 26, 9)
df['MACD'] = macd
df['MACD_signal'] = macdsignal
upper, middle, lower = talib.BBANDS(df['close'], 20)

df['Hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
df['ShootingStar'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
df['BullishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
df['BearishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']) * -1

# Original strategy definitions (strict)
def s1(i): return df['RSI'].iat[i] > df['RSI'].iat[i-1] and df['close'].iat[i] < df['close'].iat[i-1] and df['Hammer'].iat[i] > 0
def s2(i): return (df['MACD'].iat[i] - df['MACD_signal'].iat[i]) < 0 and (df['MACD'].iat[i-1] - df['MACD_signal'].iat[i-1]) > 0 and df['ShootingStar'].iat[i] < 0
def s3(i): return df['close'].iat[i-1] < lower[i-1] and df['close'].iat[i] > lower[i] and df['RSI'].iat[i] > 50 and df['BullishEngulfing'].iat[i] == 100
def s4(i): return df['ATR'].iat[i] > df['mean_ATR'].iat[i] * 1.3 and df['volume'].iat[i] > df['Volume_MA20'].iat[i] * 1.5 and df['RSI'].iat[i] < 30
def s5(i): return df['RSI'].iat[i] > 70 and df['MACD'].iat[i] < df['MACD_signal'].iat[i] and df['BearishEngulfing'].iat[i] == -100

# Combination definitions
combos = {
    "Strategy 1 + 3 (strict)": lambda i: s1(i) or s3(i),
    "Strategy 2 + 5 (strict)": lambda i: s2(i) or s5(i),
    "Strategy 2 + 4 (strict)": lambda i: s2(i) or s4(i),
    "Strategy 3 + 4 (strict)": lambda i: s3(i) or s4(i),
    "Strategy 1 + 2 + 4 (strict)": lambda i: s1(i) or s2(i) or s4(i)
}

# Backtest
results = []

for name, condition in combos.items():
    total = 0
    hit = 0
    for i in tqdm(range(2, len(df) - 1), desc=f"Backtesting {name}"):
        next_return = (df['close'].iat[i+1] - df['close'].iat[i]) / df['close'].iat[i]
        signal = condition(i)
        if signal:
            total += 1
            if (next_return > 0 and (s1(i) or s3(i) or s4(i))) or (next_return < 0 and (s2(i) or s5(i))):
                hit += 1
    acc = hit / total if total > 0 else np.nan
    results.append({"Strategy": name, "Signal Count": total, "Correct Predictions": hit, "Accuracy": f"{acc:.2%}"})

# Show results
summary_df = pd.DataFrame(results)
print("\n=== Combined Strategy Backtest Summary (Strict Conditions) ===")
print(summary_df)

Backtesting Strategy 1 + 3 (strict):   0%|          | 0/525997 [00:00<?, ?it/s]C:\Users\XIE\AppData\Local\Temp\ipykernel_16136\3651611585.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  def s3(i): return df['close'].iat[i-1] < lower[i-1] and df['close'].iat[i] > lower[i] and df['RSI'].iat[i] > 50 and df['BullishEngulfing'].iat[i] == 100
Backtesting Strategy 1 + 2 + 4 (strict): 100%|██████████| 525997/525997 [01:18<00:00, 6663.74it/s]


=== Combined Strategy Backtest Summary (Strict Conditions) ===
                      Strategy  Signal Count  Correct Predictions Accuracy
0      Strategy 1 + 3 (strict)            60                   35   58.33%
1      Strategy 2 + 5 (strict)            48                   24   50.00%
2      Strategy 2 + 4 (strict)          2710                 1490   54.98%
3      Strategy 3 + 4 (strict)          2746                 1511   55.03%
4  Strategy 1 + 2 + 4 (strict)          2710                 1490   54.98%


组合策略（单个策略已放宽）（1+3，2+5，2+4，3+4，1+2+4）

In [24]:
import pandas as pd
import numpy as np
import talib
from tqdm import tqdm

# Load data
df = pd.read_csv("BTCUSDT_1min_2024-05-01_to_2025-05-01.csv")
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index('timestamp', inplace=True)

# Technical indicators
df['RSI'] = talib.RSI(df['close'], 14)
df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], 14)
df['mean_ATR'] = df['ATR'].rolling(20).mean()
df['Volume_MA20'] = df['volume'].rolling(20).mean()
macd, macdsignal, _ = talib.MACD(df['close'], 12, 26, 9)
df['MACD'] = macd
df['MACD_signal'] = macdsignal
upper, middle, lower = talib.BBANDS(df['close'], 20)

df['Hammer'] = talib.CDLHAMMER(df['open'], df['high'], df['low'], df['close'])
df['ShootingStar'] = talib.CDLSHOOTINGSTAR(df['open'], df['high'], df['low'], df['close'])
df['BullishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
df['BearishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close']) * -1

# Strategy functions (relaxed version)
def s1(i): return df['RSI'].iat[i] > df['RSI'].iat[i-1] and df['Hammer'].iat[i] > 0
def s2(i): return df['MACD'].iat[i] < df['MACD_signal'].iat[i] and df['ShootingStar'].iat[i] < 0
def s3(i): return df['close'].iat[i] < lower[i] * 1.01 and df['RSI'].iat[i] > 45 and df['BullishEngulfing'].iat[i] == 100
def s4(i): return df['ATR'].iat[i] > df['mean_ATR'].iat[i] * 1.1 and df['volume'].iat[i] > df['Volume_MA20'].iat[i] * 1.2 and df['RSI'].iat[i] < 35
def s5(i): return df['RSI'].iat[i] > 65 and df['MACD'].iat[i] < df['MACD_signal'].iat[i] and df['BearishEngulfing'].iat[i] == -100

# Combination definitions
combos = {
    "Strategy 1 + 3": lambda i: s1(i) or s3(i),
    "Strategy 2 + 5": lambda i: s2(i) or s5(i),
    "Strategy 2 + 4": lambda i: s2(i) or s4(i),
    "Strategy 3 + 4": lambda i: s3(i) or s4(i),
    "Strategy 1 + 2 + 4": lambda i: s1(i) or s2(i) or s4(i)
}

# Backtest
results = []

for name, condition in combos.items():
    total = 0
    hit = 0
    for i in tqdm(range(2, len(df) - 1), desc=f"Backtesting {name}"):
        next_return = (df['close'].iat[i+1] - df['close'].iat[i]) / df['close'].iat[i]
        signal = condition(i)
        if signal:
            total += 1
            if (next_return > 0 and (s1(i) or s3(i) or s4(i))) or (next_return < 0 and (s2(i) or s5(i))):
                hit += 1
    acc = hit / total if total > 0 else np.nan
    results.append({"Strategy": name, "Signal Count": total, "Correct Predictions": hit, "Accuracy": f"{acc:.2%}"})

# Show results
summary_df = pd.DataFrame(results)
print("\n=== Combined Strategy Backtest Summary ===")
print(summary_df)

Backtesting Strategy 1 + 3:   0%|          | 0/525997 [00:00<?, ?it/s]C:\Users\XIE\AppData\Local\Temp\ipykernel_16136\1378666412.py:29: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  def s3(i): return df['close'].iat[i] < lower[i] * 1.01 and df['RSI'].iat[i] > 45 and df['BullishEngulfing'].iat[i] == 100
Backtesting Strategy 1 + 2 + 4: 100%|██████████| 525997/525997 [01:15<00:00, 6939.86it/s]


=== Combined Strategy Backtest Summary ===
             Strategy  Signal Count  Correct Predictions Accuracy
0      Strategy 1 + 3         20258                10263   50.66%
1      Strategy 2 + 5          1334                  782   58.62%
2      Strategy 2 + 4         12843                 6760   52.64%
3      Strategy 3 + 4         22180                11608   52.34%
4  Strategy 1 + 2 + 4         22547                11526   51.12%


确认使用策略3+4（relaxed），定义策略

### Plot Realtime result:

In [21]:
import os
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import talib
from binance.client import Client
import time
from signal_logger import SignalHistoryLogger
from prediction_logger import PredictionLogger


def fetch_ohlcv_data(symbol="BTCUSDT", interval=Client.KLINE_INTERVAL_5MINUTE, limit=1000):
    klines = client.get_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume',
        'close_time', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_volume', 'taker_buy_quote_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
    return df


def generate_strategy34_signals(df, signal_logger=None):
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['mean_ATR'] = df['ATR'].rolling(20).mean()
    df['Volume_MA20'] = df['volume'].rolling(20).mean()
    df['BullishEngulfing'] = talib.CDLENGULFING(df['open'], df['high'], df['low'], df['close'])
    _, _, df['lower'] = talib.BBANDS(df['close'], timeperiod=20)

    df['bullish_combined'] = 0
    df['bullish_trigger'] = ''
    i = -2
    ts = df.index[i]
    strategy3 = (
        df['close'].iloc[i] < df['lower'].iloc[i] * 1.01 and
        df['RSI'].iloc[i] > 45 and
        df['BullishEngulfing'].iloc[i] == 100
    )
    strategy4 = (
        df['ATR'].iloc[i] > df['mean_ATR'].iloc[i] * 1.1 and
        df['volume'].iloc[i] > df['Volume_MA20'].iloc[i] * 1.2 and
        df['RSI'].iloc[i] < 35
    )
    if strategy3 or strategy4:
        df.at[ts, 'bullish_combined'] = 1
        reasons = []
        if strategy3: reasons.append('Strategy 3')
        if strategy4: reasons.append('Strategy 4')
        df.at[ts, 'bullish_trigger'] = ' + '.join(reasons)
        print(f"[SIGNAL TRIGGERED] at {ts}: {df.at[ts, 'bullish_trigger']}")
        if signal_logger is not None:
            signal_logger.add_signal('bullish', ts, df['close'].iloc[i], df.at[ts, 'bullish_trigger'])
    return df

In [23]:
def plot_realtime_signals(df, symbol='BTCUSDT', data_range=50, output_dir='plots/', signal_logger=None):
    if signal_logger is None:
        raise ValueError("signal_logger must be provided")

    df_plot = df.iloc[-data_range:].copy()
    df_plot['hover_text'] = np.where(df_plot.index == df.index[-1], ' Latest forming candle (not evaluated)', '')
    up_signals = df_plot[df_plot['bullish_combined'] == 1]

    print(f"Plotting Real-Time: {len(up_signals)} Bullish UP signals in last {data_range} K-lines")

    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.1,
        subplot_titles=['Candlestick + MA', 'RSI', 'ATR', 'Volume'],
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )

    fig.add_trace(go.Candlestick(
        x=df_plot.index,
        open=df_plot['open'], high=df_plot['high'],
        low=df_plot['low'], close=df_plot['close'],
        name='Candlestick', increasing_line_color='green', decreasing_line_color='red'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['close'], mode='lines', name='Close Price'), row=1, col=1)

    fig.add_trace(go.Scatter(
        x=up_signals.index,
        y=up_signals['close'] * 1.005,
        mode='markers', marker=dict(symbol='triangle-up', color='green', size=10),
        name='Bullish Signal (Strategy 3+4)',
        text=up_signals['bullish_trigger'], hoverinfo='text+x+y'
    ), row=1, col=1)

    fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['RSI'], mode='lines', name='RSI'), row=2, col=1)
    fig.add_hline(y=50, line_dash='dash', line_color='black', row=2, col=1)

    fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['ATR'], mode='lines', name='ATR'), row=3, col=1)
    fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['mean_ATR'], mode='lines', name='mean_ATR', line=dict(dash='dash')), row=3, col=1)

    fig.add_trace(go.Bar(x=df_plot.index, y=df_plot['volume'], name='Volume'), row=4, col=1)
    fig.add_trace(go.Scatter(x=df_plot.index, y=df_plot['Volume_MA20'], mode='lines', name='Volume_MA20', line=dict(dash='dash')), row=4, col=1)

    fig.update_layout(
        title=f'[Strategy 3+4] Real-Time Signals for {symbol}',
        xaxis_title='Time',
        yaxis_title='Price ($)',
        xaxis_rangeslider_visible=False,
        showlegend=True,
        height=800,
        template='plotly_white'
    )

    os.makedirs(output_dir, exist_ok=True)
    html_path = os.path.join(output_dir, f'realtime_signals_{symbol}.html')
    try:
        html_content = fig.to_html(include_plotlyjs='cdn')
        html_content = html_content.replace('<head>', '<head><meta http-equiv="refresh" content="300">')
        with open(html_path, 'w') as f:
            f.write(html_content)
        print(f"Real-time plot saved: {html_path} (auto-refreshes every 5 minutes)")
    except Exception as e:
        print(f"Error saving real-time plot: {e}")

In [ ]:
# === Run Strategy 3+4 Real-Time ===
logger = PredictionLogger()
signal_logger = SignalHistoryLogger(filename="D:/My File/SMU/QF 635/MMAT/signal_history_xie.csv")
client = Client(api_key='your_api_key', api_secret='your_api_secret')

import atexit
atexit.register(lambda: logger.save_to_csv("TestLive_prediction_log.csv"))
atexit.register(lambda: signal_logger.save_to_csv("signal_history.csv"))

while True:
    try:
        df = fetch_ohlcv_data()
        df = generate_strategy34_signals(df, signal_logger=signal_logger)
        plot_realtime_signals(df, symbol="BTCUSDT", signal_logger=signal_logger)
        print("[INFO] Waiting 300 seconds for next update...")
        time.sleep(300)
    except Exception as e:
        print(f"Error in real-time loop: {e}. Retrying in 60 seconds.")
        time.sleep(60)

Plotting Real-Time: 0 Bullish UP signals in last 50 K-lines
Real-time plot saved: plots/realtime_signals_BTCUSDT.html (auto-refreshes every 5 minutes)
[INFO] Waiting 300 seconds for next update...
Plotting Real-Time: 0 Bullish UP signals in last 50 K-lines
Real-time plot saved: plots/realtime_signals_BTCUSDT.html (auto-refreshes every 5 minutes)
[INFO] Waiting 300 seconds for next update...
